In [1]:
import numpy as np
import pandas as pd
import yahoo_fin.stock_info as si
import datetime
import sqlite3 as sq

In [2]:
start = datetime.datetime(2020,1,1)
end = datetime.datetime(2023,8,4)
historical_datas = {}

In [3]:
tickers_csv = pd.read_excel('Copy of NYSE_Stocks.xlsx')
tickers_list= tickers_csv['Symbol'].tolist()

ticker_count = len(tickers_list)
print(f"Number of unique tickers: {ticker_count}")

Number of unique tickers: 8970


In [4]:
for idx, symbol in enumerate(tickers_list, start=1):  # start=1 makes idx start from 1 instead of 0
    try:
        historical_datas[symbol] = si.get_data(symbol, start_date=start, end_date=end, index_as_date=True)
        print(f"Fetching data for {symbol}: {idx}/{ticker_count}")
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        continue

Fetching data for A: 1/8970
Fetching data for AA: 2/8970
Fetching data for AAA: 3/8970
Fetching data for AAAU: 4/8970
Fetching data for AAC: 5/8970
Fetching data for AACG: 6/8970
Fetching data for AACI: 7/8970
Fetching data for AACT: 8/8970
Fetching data for AADI: 9/8970
Fetching data for AADR: 10/8970
Fetching data for AAIC: 11/8970
Fetching data for AAL: 12/8970
Fetching data for AAMC: 13/8970
Fetching data for AAME: 14/8970
Fetching data for AAN: 15/8970
Fetching data for AAOI: 16/8970
Fetching data for AAON: 17/8970
Fetching data for AAP: 18/8970
Fetching data for AAPB: 19/8970
Fetching data for AAPD: 20/8970
Fetching data for AAPL: 21/8970
Fetching data for AAPU: 22/8970
Fetching data for AAT: 23/8970
Fetching data for AAXJ: 24/8970
Fetching data for AB: 25/8970
Fetching data for ABBV: 26/8970
Fetching data for ABC: 27/8970
Fetching data for ABCB: 28/8970
Fetching data for ABCL: 29/8970
Fetching data for ABCM: 30/8970
Fetching data for ABEO: 31/8970
Fetching data for ABEQ: 32/8970

In [5]:
# Concatenate the dataframes in the dictionary
all_data = pd.concat(historical_datas.values())
all_data = all_data.reset_index().rename(columns={'index': 'date'})
cols = ['date', 'ticker'] + [col for col in all_data.columns if col not in ['date', 'ticker']]
all_data = all_data[cols]
# Check for duplicate 'ticker' columns and keep only one
if all_data.columns.duplicated().sum() > 0:
    all_data = all_data.loc[:, ~all_data.columns.duplicated()]

all_data

,date,ticker,open,high,low,close,adjclose,volume
0,2020-01-02,A,85.900002,86.349998,85.199997,85.949997,83.948044,1410500.0
1,2020-01-03,A,84.669998,85.330002,84.500000,84.570000,82.600204,1118300.0
2,2020-01-06,A,84.000000,84.820000,83.599998,84.820000,82.844376,1993200.0
3,2020-01-07,A,83.959999,85.260002,83.940002,85.080002,83.098297,1684700.0
4,2020-01-08,A,85.959999,86.470001,85.199997,85.919998,83.918747,1847600.0
...,...,...,...,...,...,...,...,...
5410964,2023-07-28,ZYXI,10.120000,10.450000,9.880000,9.890000,9.890000,328000.0
5410965,2023-07-31,ZYXI,9.850000,10.160000,9.730000,9.760000,9.760000,213200.0
5410966,2023-08-01,ZYXI,9.720000,9.720000,8.500000,8.620000,8.620000,849900.0
5410967,2023-08-02,ZYXI,8.650000,8.800000,8.200000,8.290000,8.290000,523200.0


In [6]:
# Connect to SQLite database (will be created if doesn't exist)
conn = sq.connect('stock_data.db')

# Insert data into a table called "stock_data"
all_data.to_sql('stock_data', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()